In [8]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:05:08)
SparkSession available as 'spark'.


In [9]:
sc 

In [4]:
dir(sc)

['PACKAGE_EXTENSIONS',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__enter__',
 '__exit__',
 '__format__',
 '__getattribute__',
 '__getnewargs__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accumulatorServer',
 '_active_spark_context',
 '_batchSize',
 '_callsite',
 '_checkpointFile',
 '_conf',
 '_dictToJavaMap',
 '_do_init',
 '_ensure_initialized',
 '_gateway',
 '_getJavaStorageLevel',
 '_initialize_context',
 '_javaAccumulator',
 '_jsc',
 '_jvm',
 '_lock',
 '_next_accum_id',
 '_pickled_broadcast_vars',
 '_python_includes',
 '_temp_dir',
 '_unbatched_serializer',
 'accumulator',
 'addFile',
 'addPyFile',
 'appName',
 'applicationId',
 'binaryFiles',
 'binaryRecords',
 'broadcast',
 'cancelAllJobs',
 'cancelJobGroup',
 'defaultMinPartitions',
 'defaultParallelism',
 'dump_profiles',
 'emptyRDD',
 'environment',
 'getConf',
 'getLocalProp

In [5]:
sc.appName

u'pyspark-shell'

In [6]:
sc.master

u'local[*]'

In [7]:
help(sc)

Help on SparkContext in module pyspark.context object:

class SparkContext(__builtin__.object)
 |  Main entry point for Spark functionality. A SparkContext represents the
 |  connection to a Spark cluster, and can be used to create L{RDD} and
 |  broadcast variables on that cluster.
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |      Enable 'with SparkContext(...) as sc: app(sc)' syntax.
 |  
 |  __exit__(self, type, value, trace)
 |      Enable 'with SparkContext(...) as sc: app' syntax.
 |      
 |      Specifically stop the context on exit of the with block.
 |  
 |  __getnewargs__(self)
 |  
 |  __init__(self, master=None, appName=None, sparkHome=None, pyFiles=None, environment=None, batchSize=0, serializer=PickleSerializer(), conf=None, gateway=None, jsc=None, profiler_cls=<class 'pyspark.profiler.BasicProfiler'>)
 |      Create a new SparkContext. At least the master and app name should be set,
 |      either through the named parameters here or through C{conf}.
 |   

In [8]:
# After reading the help we've decided we want to use sc.version to see what version of Spark we are running
sc.version

u'2.1.1'

In [1]:
data = xrange(1, 10001)

In [2]:
data[0]

1

In [3]:
# We can check the size of the list using the len() function
len(data)

10000

In [10]:
data = xrange(1, 1000000)
xrangeRDD = sc.parallelize(data, 8)

In [11]:
# Let's see how many partitions the RDD will be split into by using the getNumPartitions()
xrangeRDD.getNumPartitions()

8

![partitions](http://spark-mooc.github.io/web-assets/images/partitions.png)

In [13]:
def sub(value):
    """"Subtracts one from `value`.

    Args:
       value (int): A number.

    Returns:
        int: `value` minus one.
    """
    return (value - 1)

subRDD = xrangeRDD.map(sub)

In [14]:
print subRDD.count()

999999


In [16]:
subRDD.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [17]:
print xrangeRDD.count()

999999


![filter](http://spark-mooc.github.io/web-assets/images/filter.png)

In [18]:
def ten(value):
    """Return whether value is below ten.

    Args:
        value (int): A number.

    Returns:
        bool: Whether `value` is less than ten.
    """
    if (value < 10):
        return True
    else:
        return False

filteredRDD = subRDD.filter(ten)

print filteredRDD.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [19]:
lambdaRDD = subRDD.filter(lambda x: x < 10)
lambdaRDD.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [20]:
evenRDD = lambdaRDD.filter(lambda x: x % 2 == 0)
evenRDD.collect()

[0, 2, 4, 6, 8]

### ** Additional RDD actions **

In [21]:
print filteredRDD.first()

print filteredRDD.take(4)

print filteredRDD.take(12)

0
[0, 1, 2, 3]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [22]:
# Retrieve the three smallest elements
print filteredRDD.takeOrdered(3)
# Retrieve the five largest elements
print filteredRDD.top(5)

[0, 1, 2]
[9, 8, 7, 6, 5]


In [24]:
filteredRDD.takeOrdered(4, lambda s: -s)

[9, 8, 7, 6]

In [25]:
from operator import add

print filteredRDD.reduce(add)

print filteredRDD.reduce(lambda a, b: a + b)

45
45


In [25]:
print filteredRDD.takeSample(withReplacement=True, num=6)

print filteredRDD.takeSample(withReplacement=False, num=6)

[2, 3, 0, 4, 1, 7]
[7, 3, 4, 2, 5, 0]


In [29]:
print filteredRDD.takeSample(withReplacement=False, num=6, seed=500)

[0, 2, 5, 3, 6, 9]


In [30]:
repetitiveRDD = sc.parallelize([1, 2, 3, 1, 2, 3, 1, 2, 1, 2, 3, 3, 3, 4, 5, 4, 6])
print repetitiveRDD.countByValue()

defaultdict(<type 'int'>, {1: 4, 2: 4, 3: 5, 4: 2, 5: 1, 6: 1})


### **Additional RDD transformations **

In [32]:
wordsList = ['cat', 'elephant', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)

singularAndPluralWordsRDDMap = wordsRDD.map(lambda x: (x, x + 's'))
singularAndPluralWordsRDD = wordsRDD.flatMap(lambda x: (x, x + 's'))

print singularAndPluralWordsRDDMap.collect()
print singularAndPluralWordsRDD.collect()
print singularAndPluralWordsRDDMap.count()
print singularAndPluralWordsRDD.count()

[('cat', 'cats'), ('elephant', 'elephants'), ('rat', 'rats'), ('cat', 'cats')]
['cat', 'cats', 'elephant', 'elephants', 'rat', 'rats', 'cat', 'cats']
4
8


In [33]:
simpleRDD = sc.parallelize([2, 3, 4])
print simpleRDD.map(lambda x: range(1, x)).collect()
print simpleRDD.flatMap(lambda x: range(1, x)).collect()

[[1], [1, 2], [1, 2, 3]]
[1, 1, 2, 1, 2, 3]


### **  `groupByKey` and `reduceByKey` **


![reduceByKey() figure](http://spark-mooc.github.io/web-assets/images/reduce_by.png)

![groupByKey() figure](http://spark-mooc.github.io/web-assets/images/group_by.png)



In [34]:
pairRDD = sc.parallelize([('a', 1), ('a', 2), ('b', 1)])

print pairRDD.groupByKey().mapValues(lambda x: list(x)).collect()

print pairRDD.groupByKey().map(lambda (k, v): (k, sum(v))).collect()
print pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect()
print pairRDD.reduceByKey(add).collect()

[('a', [1, 2]), ('b', [1])]
[('a', 3), ('b', 1)]
[('a', 3), ('b', 1)]
[('a', 3), ('b', 1)]


### Caching RDDs and storage options 

In [27]:
filteredRDD.cache()
print filteredRDD.is_cached

True


In [36]:
print filteredRDD.toDebugString()

(8) PythonRDD[3] at collect at <ipython-input-18-870118197f8d>:17 [Memory Serialized 1x Replicated]
 |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:475 [Memory Serialized 1x Replicated]


In [37]:
filteredRDD.unpersist()

print filteredRDD.getStorageLevel()
filteredRDD.cache()
print filteredRDD.getStorageLevel()

Serialized 1x Replicated
Memory Serialized 1x Replicated


In [28]:
filteredRDD.count()

10

### DataFrames 

In [29]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

In [30]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

employeeSchema = StructType([
                       StructField('id', IntegerType(), nullable=False),
                       StructField('first_name', StringType(), nullable=False),
                       StructField('last_name', StringType(), nullable=False),
                       StructField('email', StringType(), nullable=False),
                       StructField('phone', StringType(), nullable=False),
                       StructField('department', StringType(), nullable=False)
                      ])

In [31]:
employeeSchema

StructType(List(StructField(id,IntegerType,false),StructField(first_name,StringType,false),StructField(last_name,StringType,false),StructField(email,StringType,false),StructField(phone,StringType,false),StructField(department,StringType,false)))

In [32]:
from pyspark.sql import Row
employeeData = sc.parallelize([
                                Row(1,'John','Doe','john.doe@company.com','555-12345', 'Sales'),
                                Row(2,'Johnny','Wishbone','johnny.wishbone@company.com','555-01534','IT'),
                                Row(3,'Max','Mustermann','max.mustermann@company.com','555-45678','IT'),
                                Row(4,'Helga','Musterfrau','helga.musterfrau@company.com','555-0963','Business'),
                                Row(5,'George','Schmidt','george.schmidt@company.com','555-67232','Business'),
                                Row(6,'Bob','Moore','bob.moore@company.com','555-78123','Sales'),
                                Row(6,'Alex','Warny','alex.warny@company.com','555-4567','Operations'),
                                Row(6,'Fred','Bernoulli','fred.bernoulli@company.com','555-0984','Operations'),
                                Row(6,'Martha','Richards','martha.richards@company.com','555-45123','Acquisitions'),
                                Row(6,'Helena','Cartier','helena.cartier@company.com','555-34908','Acquisitions')
                             ])

In [33]:
employeeData

ParallelCollectionRDD[21] at parallelize at PythonRDD.scala:475

In [34]:
employeeDF = sqlCtx.createDataFrame(employeeData, employeeSchema)

In [35]:
selected = employeeDF.select('first_name','last_name')
selected.show()

+----------+----------+
|first_name| last_name|
+----------+----------+
|      John|       Doe|
|    Johnny|  Wishbone|
|       Max|Mustermann|
|     Helga|Musterfrau|
|    George|   Schmidt|
|       Bob|     Moore|
|      Alex|     Warny|
|      Fred| Bernoulli|
|    Martha|  Richards|
|    Helena|   Cartier|
+----------+----------+



In [46]:
employeeDF.select('email').take(2)

[Row(email=u'john.doe@company.com'), Row(email=u'johnny.wishbone@company.com')]

In [47]:
import pyspark.sql.functions as func

employeeDF.select(func.col('department').alias('group')).show()

+------------+
|       group|
+------------+
|       Sales|
|          IT|
|          IT|
|    Business|
|    Business|
|       Sales|
|  Operations|
|  Operations|
|Acquisitions|
|Acquisitions|
+------------+



In [48]:
employeeDF.filter(func.col('department') == 'Business').groupBy('email').count().show()

+--------------------+-----+
|               email|count|
+--------------------+-----+
|helga.musterfrau@...|    1|
|george.schmidt@co...|    1|
+--------------------+-----+



### Yet another example

In [14]:
import pandas as pd

comments_json = './results-20151009-103553.json' 
with open(comments_json) as cmnts:              
    all_entries = cmnts.readlines()             

parsable_data = "[" + ','.join(all_entries).replace('\n','') + "]" 

In [15]:
comments = pd.read_json(parsable_data, orient='columns',typ='frame',convert_dates=['created'])

In [16]:
sparkDF = sqlCtx.createDataFrame(comments)

In [17]:
sparkDF.printSchema()

root
 |-- author: string (nullable = true)
 |-- avg_score: long (nullable = true)
 |-- comment: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- name: string (nullable = true)
 |-- subr: string (nullable = true)
 |-- ups: long (nullable = true)
 |-- url: string (nullable = true)



In [18]:
sparkDF.write.parquet("./results-20151009-103553.parquet")

In [19]:
parquet = sqlCtx.read.parquet("./results-20151009-103553.parquet")

In [60]:
pandas_df = parquet.where(func.col('avg_score') > 10).toPandas()

In [61]:
pandas_df

,author,avg_score,comment,controversiality,created,distinguished,downs,name,subr,ups,url
0,Keundrum,3877,I don't know why they would possibly tell peop...,0,1439958069000000000,None,0,t1_cu7x0ut,worldnews,3877,http://reddit.com/r/worldnews/comments/3hidw9/...
1,McBjure,13,He wont earn shit because it's against youtube...,0,1439230955000000000,None,0,t1_cty7slu,videos,13,http://reddit.com/r/videos/comments/3ggt2v/c/c...
2,Egsession,37,They weren't dating that long though -- not ev...,0,1438453918000000000,None,0,t1_cto3vte,movies,37,http://reddit.com/r/movies/comments/3fe8ru/c/c...
3,fresh_prince_,13,kek,0,1439654076000000000,None,0,t1_cu3ync9,DotA2,13,http://reddit.com/r/DotA2/comments/3h30x7/c/cu...
4,djohn_14,11,"I think DS2's DLC had AMAZING music, especiall...",0,1438713706000000000,None,0,t1_ctrdvzs,Games,11,http://reddit.com/r/Games/comments/3fr6dt/c/ct...
5,MissyPie,30,"Thank you! ^ ^\n\nI'm sorry, my heart belongs ...",0,1440596771000000000,None,0,t1_cug8byb,anime,30,http://reddit.com/r/anime/comments/3igs0f/c/cu...
6,Nilwx,23,"""Spaceship""",0,1439068042000000000,None,0,t1_ctw8zz5,GlobalOffensive,23,http://reddit.com/r/GlobalOffensive/comments/3...
7,Mike_Gainer,78,Mastodon: Pendulous Skin\nhttps://www.youtube....,0,1440167085000000000,None,0,t1_cuao760,Music,78,http://reddit.com/r/Music/comments/3hu1mf/c/cu...
8,nopooq,11,Yes! This happens SO often. Thank you for shar...,0,1440174263000000000,None,0,t1_cuaswna,AskReddit,11,http://reddit.com/r/AskReddit/comments/3hu28g/...
9,LutherDingle,11,The article characterizes Trump as an outsider...,0,1438699354000000000,None,0,t1_ctr482t,politics,11,http://reddit.com/r/politics/comments/3fqr96/c...


### User defined functions

In [37]:
def dt_conversion(timestamp):
    return datetime.fromtimestamp(timestamp/ 1000000000)


In [38]:
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as func
from datetime import datetime

dt_conv = func.udf(dt_conversion, TimestampType())
new_df = parquet.withColumn('created_h', dt_conv(parquet.created))

In [39]:
new_df.select('created', 'created_h').show()

+-------------------+--------------------+
|            created|           created_h|
+-------------------+--------------------+
|1439958069000000000|2015-08-19 07:21:...|
|1439958091000000000|2015-08-19 07:21:...|
|1439776080000000000|2015-08-17 04:48:...|
|1439776681000000000|2015-08-17 04:58:...|
|1439533443000000000|2015-08-14 09:24:...|
|1439533453000000000|2015-08-14 09:24:...|
|1440175759000000000|2015-08-21 19:49:...|
|1439230955000000000|2015-08-10 21:22:...|
|1439231094000000000|2015-08-10 21:24:...|
|1440518065000000000|2015-08-25 18:54:...|
|1439747290000000000|2015-08-16 20:48:...|
|1439747532000000000|2015-08-16 20:52:...|
|1439747786000000000|2015-08-16 20:56:...|
|1439118706000000000|2015-08-09 14:11:...|
|1439119438000000000|2015-08-09 14:23:...|
|1438453918000000000|2015-08-01 21:31:...|
|1439285611000000000|2015-08-11 12:33:...|
|1439286052000000000|2015-08-11 12:40:...|
|1439286791000000000|2015-08-11 12:53:...|
|1439855123000000000|2015-08-18 02:45:...|
+----------

### SQL query example

In [83]:
import urllib
f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", 
                    "kddcup.data_10_percent.gz")


data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file).cache()

KeyboardInterrupt: 

In [54]:
from pyspark.sql import Row

csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

In [55]:
interactions_df = sqlCtx.createDataFrame(row_data)
interactions_df.registerTempTable("interactions")
interactions_df.printSchema()

root
 |-- dst_bytes: long (nullable = true)
 |-- duration: long (nullable = true)
 |-- flag: string (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- src_bytes: long (nullable = true)



In [56]:
# Select tcp network interactions with more than 1 second duration and no transfer from destination
tcp_interactions = sqlCtx.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND 
    duration > 1000 AND dst_bytes = 0
""")
tcp_interactions.show()

+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
|    5039|        0|
|    5062|        0|
|    5041|        0|
|    5056|        0|
|    5064|        0|
|    5043|        0|
|    5061|        0|
|    5049|        0|
|    5061|        0|
|    5048|        0|
|    5047|        0|
|    5044|        0|
|    5063|        0|
|    5068|        0|
|    5062|        0|
+--------+---------+
only showing top 20 rows



In [57]:
tcp_interactions 

DataFrame[duration: bigint, dst_bytes: bigint]

In [17]:
from time import time

t0 = time()
interactions_df.select("protocol_type", "duration", "dst_bytes").groupBy("protocol_type").count().show()
tt = time() - t0

print "Query performed in {} seconds".format(round(tt,3))

NameError: name 'interactions_df' is not defined

In [16]:
t0 = time()

{interactions_df.select("protocol_type", "duration", "dst_bytes")
    .filter(interactions_df.duration>1000)
    .filter(interactions_df.dst_bytes==0)
    .groupBy("protocol_type")
    .count()
    .show()
}  
tt = time() - t0

print "Query performed in {} seconds".format(round(tt,3))

NameError: name 'time' is not defined

In [59]:
def get_label_type(label):
    if label!="normal.":
        return "attack"
    else:
        return "normal"

row_labeled_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5]),
    label=get_label_type(p[41])
    )
)
interactions_labeled_df = sqlCtx.createDataFrame(row_labeled_data)

### GraphFrames

In [88]:
from graphframes import * 

vertices = sqlCtx.createDataFrame([
  ("1", "Alex", 28, "M"),
  ("2", "Emeli", 28, "F"),
  ("3", "Natasha", 27, "F"),
  ("4", "Pavel", 30, "M"),
  ("5", "Oleg", 35, "M"),
  ("6", "Ivan", 30, "M"),
  ("7", "Ilya", 29, "M")], 
    ["id", "name", "age", "gender"])

edges = sqlCtx.createDataFrame([
  ("1", "2", "friend"),
  ("2", "1", "friend"),
  ("1", "3", "friend"),
  ("3", "1", "friend"),
  ("1", "4", "friend"),
  ("4", "1", "friend"),
  ("2", "3", "friend"), 
  ("3", "2", "friend"),
  ("2", "5", "friend"),
  ("5", "2", "friend"),
  ("3", "4", "friend"),
  ("4", "3", "friend"),
  ("3", "5", "friend"),
  ("5", "3", "friend"),
  ("3", "6", "friend"),
  ("6", "3", "friend"),
  ("3", "7", "friend"),
  ("7", "3", "friend")
], ["src", "dst", "relationship"])

g = GraphFrame(vertices, edges)


In [90]:
g.vertices.filter("age > 30").show()

+---+----+---+------+
| id|name|age|gender|
+---+----+---+------+
|  5|Oleg| 35|     M|
+---+----+---+------+



In [91]:
g.inDegrees.filter("inDegree > 2").show()

+---+--------+
| id|inDegree|
+---+--------+
|  3|       6|
|  1|       3|
|  2|       3|
+---+--------+



In [92]:
g.find("(a)-[e]->(b); (b)-[e2]->(a)").show(15)

+----------------+------------+----------------+------------+
|               a|           e|               b|          e2|
+----------------+------------+----------------+------------+
|   [1,Alex,28,M]|[1,4,friend]|  [4,Pavel,30,M]|[4,1,friend]|
|  [4,Pavel,30,M]|[4,1,friend]|   [1,Alex,28,M]|[1,4,friend]|
|[3,Natasha,27,F]|[3,2,friend]|  [2,Emeli,28,F]|[2,3,friend]|
|  [2,Emeli,28,F]|[2,1,friend]|   [1,Alex,28,M]|[1,2,friend]|
|  [2,Emeli,28,F]|[2,5,friend]|   [5,Oleg,35,M]|[5,2,friend]|
|[3,Natasha,27,F]|[3,5,friend]|   [5,Oleg,35,M]|[5,3,friend]|
|   [1,Alex,28,M]|[1,3,friend]|[3,Natasha,27,F]|[3,1,friend]|
|[3,Natasha,27,F]|[3,1,friend]|   [1,Alex,28,M]|[1,3,friend]|
|   [5,Oleg,35,M]|[5,3,friend]|[3,Natasha,27,F]|[3,5,friend]|
|  [2,Emeli,28,F]|[2,3,friend]|[3,Natasha,27,F]|[3,2,friend]|
|[3,Natasha,27,F]|[3,7,friend]|   [7,Ilya,29,M]|[7,3,friend]|
|  [4,Pavel,30,M]|[4,3,friend]|[3,Natasha,27,F]|[3,4,friend]|
|   [6,Ivan,30,M]|[6,3,friend]|[3,Natasha,27,F]|[3,6,friend]|
|   [1,A

In [93]:
motifs = g.find("(A)-[]->(B); (B)-[]->(C)")
motifs.show(15)

+----------------+----------------+----------------+
|               A|               B|               C|
+----------------+----------------+----------------+
|   [7,Ilya,29,M]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
|   [5,Oleg,35,M]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
|   [6,Ivan,30,M]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
|   [1,Alex,28,M]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
|  [4,Pavel,30,M]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
|  [2,Emeli,28,F]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
|[3,Natasha,27,F]|   [7,Ilya,29,M]|[3,Natasha,27,F]|
|[3,Natasha,27,F]|   [5,Oleg,35,M]|[3,Natasha,27,F]|
|  [2,Emeli,28,F]|   [5,Oleg,35,M]|[3,Natasha,27,F]|
|[3,Natasha,27,F]|   [6,Ivan,30,M]|[3,Natasha,27,F]|
|[3,Natasha,27,F]|   [1,Alex,28,M]|[3,Natasha,27,F]|
|  [4,Pavel,30,M]|   [1,Alex,28,M]|[3,Natasha,27,F]|
|  [2,Emeli,28,F]|   [1,Alex,28,M]|[3,Natasha,27,F]|
|[3,Natasha,27,F]|  [4,Pavel,30,M]|[3,Natasha,27,F]|
|   [1,Alex,28,M]|  [4,Pavel,30,M]|[3,Natasha,27,F]|
+----------------+----------------+-----------

In [94]:
motifs = g.find("(A)-[]->(B); (B)-[]->(C)").filter("A.id != C.id")
motifs.show(15)

+--------------+----------------+----------------+
|             A|               B|               C|
+--------------+----------------+----------------+
| [5,Oleg,35,M]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
| [6,Ivan,30,M]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
| [1,Alex,28,M]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
|[4,Pavel,30,M]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
|[2,Emeli,28,F]|[3,Natasha,27,F]|   [7,Ilya,29,M]|
|[2,Emeli,28,F]|   [5,Oleg,35,M]|[3,Natasha,27,F]|
|[4,Pavel,30,M]|   [1,Alex,28,M]|[3,Natasha,27,F]|
|[2,Emeli,28,F]|   [1,Alex,28,M]|[3,Natasha,27,F]|
| [1,Alex,28,M]|  [4,Pavel,30,M]|[3,Natasha,27,F]|
| [5,Oleg,35,M]|  [2,Emeli,28,F]|[3,Natasha,27,F]|
| [1,Alex,28,M]|  [2,Emeli,28,F]|[3,Natasha,27,F]|
| [7,Ilya,29,M]|[3,Natasha,27,F]|   [5,Oleg,35,M]|
| [6,Ivan,30,M]|[3,Natasha,27,F]|   [5,Oleg,35,M]|
| [1,Alex,28,M]|[3,Natasha,27,F]|   [5,Oleg,35,M]|
|[4,Pavel,30,M]|[3,Natasha,27,F]|   [5,Oleg,35,M]|
+--------------+----------------+----------------+
only showing top 15 rows



In [97]:
results = g.triangleCount()
results.show()

+-----+---+-------+---+------+
|count| id|   name|age|gender|
+-----+---+-------+---+------+
|    0|  7|   Ilya| 29|     M|
|    3|  3|Natasha| 27|     F|
|    1|  5|   Oleg| 35|     M|
|    0|  6|   Ivan| 30|     M|
|    2|  1|   Alex| 28|     M|
|    1|  4|  Pavel| 30|     M|
|    2|  2|  Emeli| 28|     F|
+-----+---+-------+---+------+



In [102]:
results = g.pageRank(resetProbability=0.15, maxIter=10)
results.vertices.show()

+---+-------+---+------+----------+--------+
| id|   name|age|gender|university|pagerank|
+---+-------+---+------+----------+--------+
|  1|   Alex| 28|     M|      MIPT|    0.15|
|  3|Natasha| 27|     F|     SPbSU|    0.15|
|  2|  Emeli| 28|     F|      MIPT| 0.21375|
|  4|  Pavel| 30|     M|      MIPT| 0.21375|
|  7|   Ilya| 29|     M|       MSU|  0.1925|
|  6|   Ivan| 30|     M|       MSU|  0.1925|
|  5|   Oleg| 35|     M|      MIPT|  0.1925|
+---+-------+---+------+----------+--------+



### new graph

In [99]:
vertices = sqlCtx.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = sqlCtx.createDataFrame([
  ("1", "2", "friend"),
  ("1", "4", "friend"),
  ("3", "5", "friend"),
  ("3", "6", "friend"),
  ("3", "7", "friend")
], ["src", "dst", "type"])

g = GraphFrame(vertices, edges)

In [101]:
sc.setCheckpointDir("/tmp")
result = g.connectedComponents()
result.select("id", "component").orderBy("component").show()

+---+-------------+
| id|    component|
+---+-------------+
|  3|  25769803776|
|  7|  25769803776|
|  5|  25769803776|
|  6|  25769803776|
|  2|1236950581248|
|  4|1236950581248|
|  1|1236950581248|
+---+-------------+

